<a href="https://colab.research.google.com/github/Mateo755/UAV_ML_FDI/blob/main/FDI_UAV_Optuna_(Colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UAV Propeller Fault Detection System (Parrot Bebop 2)

This project focuses on the development and validation of fault detection and isolation (FDI) methods for the propulsion system of the **Parrot Bebop 2** unmanned aerial vehicle (UAV). The analysis utilizes inertial sensor data (accelerometer and gyroscope) collected during real-world flight experiments.

### Research Problem
The primary objective is to classify the technical state of the propellers based on vibration signals. We analyze various fault scenarios across four rotors (A, B, C, D), distinguishing between nominal states and specific defects such as chipped edges or bent blades.

### Methodology
This notebook compares two signal processing approaches:
1.  **Time Domain Analysis**
2.  **Frequency Domain Analysis**

Experiment tracking and performance visualization are managed via **Weights & Biases (W&B)**.

# 1. Environment Setup & Global Configuration
Installation of necessary libraries (Weights & Biases for experiment tracking) and importing standard data science modules.

In [ ]:
!pip install wandb
!pip install optuna

In [ ]:
import os
import optuna
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adagrad, Adadelta, Adamax, Nadam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import CategoricalAccuracy
import traceback

In [ ]:
import wandb
wandb.login()

# 2. Exploratory Data Analysis
Initial inspection of the accelerometer and gyroscope data from the UAV. We examine the column structure.

## 2.1 Time domain

In [ ]:
!unzip -q /content/Normalized_data.zip

In [ ]:
df_time = pd.read_csv('/content/Normalized_data/Bebop2_16g_1kdps_normalized_0000.csv')
df_time

In [ ]:
df_time.info()

## 2.2 Frequency Domain

In [ ]:
!unzip -q /content/FFT_data.zip

In [ ]:
df_freq = pd.read_csv('/content/FFT_data/128_Hann_20_52/Bebop2_16g_FFT_ACCEL_128_Hann_20_52_0000.csv', header=None)
df_freq

In [ ]:
df_freq.info()

# 3. Fault Scenario Mapping

The Bebop 2 flight data is labeled using a 4-digit code (e.g., `1022`), defining the state of each propeller (A, B, C, D):
* **0**: Nominal (Functional propeller).
* **1**: Fault Type I (e.g., chipped edge).
* **2**: Fault Type II (e.g., bent tip/severe damage).

Below, we define the mapping of these physical scenarios to model class labels. Depending on the diagnostic granularity required, the problem can be framed as a **5-class problem** (aggregated by the number of faults) or a **20-class problem** (precise fault configuration).

In [ ]:
# Select Experiment Mode
CLASS_MODE = "5class"   # Options: "5class" (aggregated) or "20class" (precise diagnosis)

In [ ]:
# Precise mapping for 20 unique scenarios (Parrot Bebop 2)
scenario_to_class_20 = {
    "0000": 0,  # Nominal state
    "1000": 1, "0100": 2, "0010": 3, "0001": 4,     # Single faults (Type 1)
    "2000": 5, "0200": 6, "0020": 7, "0002": 8,     # Single faults (Type 2)
    "1100": 9, "1020": 10, "1002": 11, "0120": 12, "0102": 13, "0022": 14, # Dual faults
    "1120": 15, "1102": 16, "1022": 17, "0122": 18, # Triple faults
    "1122": 19, # All propellers faulty
}

# Simplified mapping (Number of faulty rotors)
scenario_to_class_5 = {
    "0000": 0,
    "1000": 1, "0100": 1, "0010": 1, "0001": 1,
    "2000": 1, "0200": 1, "0020": 1, "0002": 1,
    "1100": 2, "1020": 2, "1002": 2, "0120": 2, "0102": 2, "0022": 2,
    "1120": 3, "1102": 3, "1022": 3, "0122": 3,
    "1122": 4,
}

if CLASS_MODE == "5class":
    scenario_to_class = scenario_to_class_5
elif CLASS_MODE == "20class":
    scenario_to_class = scenario_to_class_20
else:
    raise ValueError("Invalid CLASS_MODE selected.")

NUM_CLASSES = len(set(scenario_to_class.values()))
print(f"Experiment Mode: {CLASS_MODE} | Total Classes: {NUM_CLASSES}")

# 3. Time Domain Analysis, data preparation

In this section, we process normalized time-series signals from the accelerometers and gyroscopes. Since the data represents a continuous flight stream, we apply a **sliding window** technique to segment the signal into fixed-length samples (e.g., 256 measurement points).



## 3.1 Data Segmentation

In [ ]:
# Path to normalized time-domain data
DATA_DIR_TIME = r"/content/Normalized_data"
SAMPLE_SIZE = 8             # Window length
N_FEATURES = 24               # Input channels (e.g., 3-axis accel + 3-axis gyro per sensor)
SENSOR_MODE = "both"          # "accel" + "gyro"

In [ ]:
def make_windows_from_df(df: pd.DataFrame, sample_size: int):
    """
    Segments time-series data into non-overlapping windows.
    """
    data = df.values.astype("float32")
    n_total = len(data)
    n_windows = n_total // sample_size
    if n_windows == 0:
        return np.empty((0, sample_size, data.shape[1]), dtype="float32")
    data = data[:n_windows * sample_size]
    windows = data.reshape(n_windows, sample_size, data.shape[1])
    return windows

### Processing Pipeline
The following loop iterates through all normalized CSV files in time domain. For each file, it:
1.  Extracts the scenario code (e.g., `0000`) from the filename.
2.  Checks if the scenario exists in our defined class mapping.
3.  Loads the data and applies the sliding window segmentation.
4.  Accumulates the processed windows (`X`) and corresponding labels (`y`) into a single dataset.

In [ ]:
# Data pass through window function

X_list = []
y_list = []

norm_data_files_pattern = os.path.join(DATA_DIR_TIME, "Bebop2_16g_1kdps_normalized_*.csv")

for path in glob.glob(norm_data_files_pattern):
    fname = os.path.basename(path)
    # ostatni fragment po "_" to kod scenariusza, np. "0000"
    scenario = os.path.splitext(fname)[0].split("_")[-1]

    if scenario not in scenario_to_class:
        print(f"Pomijam {fname} – scenariusz {scenario} nie jest w mapowaniu.")
        continue

    label = scenario_to_class[scenario]

    df = pd.read_csv(path)

    # jeżeli w pliku są inne kolumny niż 24 sensory, tu można wybrać tylko potrzebne:
    # df = df[["A_aX","A_aY",...,"D_gZ"]]

    windows = make_windows_from_df(df, SAMPLE_SIZE)
    if windows.shape[0] == 0:
        print(f"Za mało danych w {fname} na choć jedno okno, pomijam.")
        continue

    X_list.append(windows)
    y_list.append(np.full((windows.shape[0],), label, dtype="int32"))

X = np.concatenate(X_list, axis=0)  # (N, SAMPLE_SIZE, 24)
y = np.concatenate(y_list, axis=0)  # (N,)

print("X shape:", X.shape)
print("y shape:", y.shape, "unikalne etykiety:", np.unique(y))

input_shape = (SAMPLE_SIZE, N_FEATURES)
print("input_shape modelu:", input_shape)

# 4. Frequency Domain Analysis (FFT)

Mechanical faults in rotating components (such as propellers) generate distinct vibration signatures that are often most discernible in the frequency spectrum.

In this experiment, we utilize data pre-processed via **Fast Fourier Transform (FFT)** using a Hann window to mitigate spectral leakage, which is already done in repo. The input features are vectors of spectral coefficients for each sensor axis.

## 4.1 Spectral Data Structure
The FFT files contain metadata within their filenames (window length, window type, frequency range). The following code parses this information and loads the corresponding spectral coefficients.

In [ ]:
# Configuration for FFT Data
FFT_ROOT      = "FFT_data"
FFT_CONFIG    = "128_Hann_20_52"     # Specific window/range configuration
SENSOR_MODE   = "both"               # "accel", "gyro", or "both"
SAMPLING_RATE = 500.0                # Sampling rate for Bebop 2 inertial sensors

# liczba osi na jeden typ czujnika
N_AXES_SINGLE = 12
N_AXES = 12 if SENSOR_MODE in ("accel", "gyro") else 24

fft_dir = os.path.join(FFT_ROOT, FFT_CONFIG)

In [ ]:
def print_fft_info(fft_dir, sampling_rate=500.0):
    """
    Extracts FFT parameters encoded in the data folder name and converts them
    into physical frequency values.

    The function assumes the folder name follows the format:
    'WindowLength_WindowType_StartBin_StopBin' (e.g., '128_Hann_20_52').
    """

    folder_name = os.path.basename(os.path.normpath(fft_dir))
    parts = folder_name.split("_")          # np. ["128","Hann","20","52"]
    measuringWindowLength = int(parts[0])   # 128
    rangeStart = int(parts[-2])             # 20
    rangeStop  = int(parts[-1])             # 52

    freq_res = sampling_rate / measuringWindowLength
    f_start  = (rangeStart - 1) * freq_res
    f_stop   = rangeStop * freq_res

    print(f"Folder FFT: {folder_name}")
    print(f"measuringWindowLength = {measuringWindowLength}")
    print(f"Zakres binów: {rangeStart}–{rangeStop}")
    print(f"Rozdzielczość częstotliwości: {freq_res:.3f} Hz")
    print(f"Zakres częstotliwości: {f_start:.1f} Hz – {f_stop:.1f} Hz")

    return measuringWindowLength, rangeStart, rangeStop, freq_res, f_start, f_stop

In [ ]:
measuringWindowLength, rangeStart, rangeStop, freq_res, f_start, f_stop = print_fft_info(fft_dir, SAMPLING_RATE)

### File Discovery & Categorization
This block scans the directory for all CSV files and organizes them into dictionaries based on the sensor type (**ACCEL** vs. **GYRO**). It parses the filename to extract the specific fault scenario code (e.g., `0000`, `1022`), using it as a key for quick lookup during the data loading phase.

In [ ]:
# Scan directory and map file paths to scenarios based on sensor type (ACCEL/GYRO)

all_files = glob.glob(os.path.join(fft_dir, "*.csv"))

accel_files = {}  # scenario -> ścieżka
gyro_files  = {}

for path in all_files:
    fname = os.path.basename(path)
    parts = fname.split("_")
    # przykład: Bebop2_16g_FFT_ACCEL_128_Hann_20_52_0000.csv
    # indeksy:   0      1   2   3     4    5    6   7   8
    sensor_type = parts[3]              # "ACCEL" albo "GYRO"
    scenario    = os.path.splitext(parts[-1])[0]  # "0000" itd.

    if sensor_type == "ACCEL":
        accel_files[scenario] = path
    elif sensor_type == "GYRO":
        gyro_files[scenario] = path

print("Znaleziono ACCEL dla scenariuszy:", sorted(accel_files.keys()))
print("Znaleziono GYRO  dla scenariuszy:", sorted(gyro_files.keys()))


### Data Loading & Sensor Fusion
In this step, we aggregate the spectral data based on the selected `SENSOR_MODE`.
* **Accel/Gyro:** Loads only the specified sensor data.
* **Both:** Loads both accelerometer and gyroscope files for the same scenario, verifying consistency, and concatenates them along the feature axis to create a unified feature vector (e.g., 24 input channels).

In [ ]:
def load_fft_file(path, n_axes=N_AXES_SINGLE):
    """
    Loads spectral data from a CSV file and reshapes it into a 3D tensor.

    The function reads a flat CSV (assuming no header), calculates the number
    of frequency bins based on the total columns and specified axes, and
    restructures the data.
    """

    # jeśli okaże się, że plik ma nagłówek – zmień na header=0
    df = pd.read_csv(path, header=None)
    data = df.values.astype("float32")   # (n_okien, n_features)
    n_features = data.shape[1]

    if n_features % n_axes != 0:
        raise ValueError(f"{os.path.basename(path)}: {n_features} kolumn "
                         f"nie dzieli się przez {n_axes} osi.")

    n_freq_bins = n_features // n_axes
    data_3d = data.reshape(-1, n_freq_bins, n_axes)  # (n_okien, n_freq_bins, n_axes)
    return data_3d, n_freq_bins

In [ ]:
X_list = []
y_list = []
n_freq_bins_global = None

for scenario, label in scenario_to_class.items():
    cur_X = None

    if SENSOR_MODE == "accel":
        path = accel_files.get(scenario)
        if path is None:
            print(f"[ACCEL] brak pliku dla scenariusza {scenario}, pomijam.")
            continue
        accel_data, n_freq_bins = load_fft_file(path)
        cur_X = accel_data   # (n_okien, n_freq_bins, 12)

    elif SENSOR_MODE == "gyro":
        path = gyro_files.get(scenario)
        if path is None:
            print(f"[GYRO] brak pliku dla scenariusza {scenario}, pomijam.")
            continue
        gyro_data, n_freq_bins = load_fft_file(path)
        cur_X = gyro_data    # (n_okien, n_freq_bins, 12)

    elif SENSOR_MODE == "both":
        path_a = accel_files.get(scenario)
        path_g = gyro_files.get(scenario)
        if path_a is None or path_g is None:
            print(f"[BOTH] brak ACCEL lub GYRO dla {scenario}, pomijam.")
            continue

        accel_data, n_freq_bins_a = load_fft_file(path_a, n_axes=N_AXES_SINGLE)
        gyro_data,  n_freq_bins_g = load_fft_file(path_g, n_axes=N_AXES_SINGLE)

        if accel_data.shape[0] != gyro_data.shape[0] or n_freq_bins_a != n_freq_bins_g:
            raise ValueError(f"Niezgodne rozmiary ACCEL/GYRO dla scenariusza {scenario}")

        # sklejanie po osi „kanałów”: 12 (ACCEL) + 12 (GYRO) = 24
        cur_X = np.concatenate([accel_data, gyro_data], axis=-1)  # (..., n_freq_bins, 24)
        n_freq_bins = n_freq_bins_a

    else:
        raise ValueError("SENSOR_MODE musi być 'accel', 'gyro' albo 'both'")

    # ustaw / sprawdź globalną liczbę binów
    if n_freq_bins_global is None:
        n_freq_bins_global = n_freq_bins
    elif n_freq_bins_global != n_freq_bins:
        raise ValueError("Różne n_freq_bins między plikami, coś jest nie tak.")

    X_list.append(cur_X)
    y_list.append(np.full((cur_X.shape[0],), label, dtype="int32"))

# Sklejenie wszystkiego
X = np.concatenate(X_list, axis=0)   # (N, n_freq_bins, N_AXES)
y = np.concatenate(y_list, axis=0)   # (N,)

print("X shape:", X.shape)
print("y shape:", y.shape, "unikalne etykiety:", np.unique(y))

n_freq_bins = n_freq_bins_global
input_shape = (n_freq_bins, N_AXES)
print("input_shape modelu:", input_shape)


# 5. OPTUNA